In [ ]:
USERNAME = ""
PASSWORD = ""
TO_USER = ""
AMOUNT = ""
is_signed = False

In [ ]:
#!pip install solid-file
#!pip install web3

In [ ]:
# imports
from solid.solid_api import SolidAPI
from solid.auth import Auth
from getpass import getpass
from web3 import Web3, HTTPProvider
from web3.middleware import geth_poa_middleware
import json
import io

In [ ]:
#Nothing

# Welcome to the Papayaverse Demo Notebook

INSERT A BUNCH OF INTRODUCTORY TEXT

## Login

Please login to solidcommunity.net

In [ ]:
# LOGIN MODULE
#USERNAME = input('Please enter your  solid community user name:')
#PASSWORD = getpass('Please enter your solid community password:')
IDP = 'https://solidcommunity.net'
auth = Auth()
solid_connection = SolidAPI(auth)
auth.login(IDP, USERNAME, PASSWORD)


## Wallet


In [ ]:
## Setup for Web3
web3provider = 'https://goerli.infura.io/v3/85e35e212e7c431a838571e469b3c64b'
web3_connection = Web3(HTTPProvider(web3provider))
print ("Latest Ethereum block number", web3_connection.eth.blockNumber)
web3_connection.middleware_onion.inject(geth_poa_middleware, layer=0)
if web3_connection.isConnected(): # verify that the connection worked
    print('Now Connected to the Goerli Network!')
else :
    print('An Error Occurred ;-;')

In [ ]:
## Find Existing Wallet
wallet_folder = 'https://' + USERNAME + '.solidcommunity.net/public/wallet/'
private_wallet_folder = 'https://' + USERNAME + '.solidcommunity.net/private/wallet/'
required_files = ['provider.md', 'wallet_address.md']
criterion = all([solid_connection.item_exists(wallet_folder + i) for i in required_files])
if criterion: # Does the user already have a wallet?
    wallet_address_file_name = wallet_folder + 'wallet_address.md'
    wallet_address_resp = solid_connection.get(wallet_address_file_name)
    wallet_address = wallet_address_resp.text
    print('Found Existing Wallet Address' + wallet_address)
## Or Create One
else :
    my_account = web3_connection.eth.account.create(USERNAME)
    wallet_address = my_account._address
    print('Created New Wallet for You ' + wallet_address)
    ## Save wallet information if new wallet
    def save_wallet_info(file_string, file_nm):
        f = io.BytesIO(file_string.encode('UTF-8'))
        file_name = file_nm
        solid_connection.put_file(file_name, f, 'text/markdown')
    # Save provider
    save_wallet_info(web3provider, wallet_folder + 'provider.md')
    # Save wallet address
    save_wallet_info(wallet_address, wallet_folder + 'wallet_address.md')
    # Save private key
    private_key = my_account._private_key.hex()
    save_wallet_info(private_key, private_wallet_folder + 'private_key.md')
## And display the balance
balance = web3_connection.eth.get_balance(account = wallet_address) # gets balance in wei
print('Your Balance is ' + str(Web3.fromWei(balance, 'ether')) + ' ether')

## Transactions

Insert some text here about transactions

### Send Eth

In [ ]:
# Inputs
#TO_USER = input("Enter the username to whom you want to send eth ")
#AMOUNT = (input("Enter the amount you want to send (in ether) "))

In [ ]:
# Get Private Key
private_key_file_name = private_wallet_folder + 'private_key.md'
private_key = solid_connection.get(private_key_file_name).text

In [ ]:
# Get TO_ADDRESS FROM USER
TO_ADDRESS = solid_connection.get('https://' + TO_USER + '.solidcommunity.net/public/wallet/wallet_address.md').text

In [ ]:
print('Wallet Address of recipient is ' + TO_ADDRESS)

In [ ]:
transaction = {
        "nonce": web3_connection.eth.get_transaction_count(wallet_address),
        "gas": 21000,
        "to": TO_ADDRESS,
        "value": web3_connection.toWei(AMOUNT, "ether"),
        "gasPrice" : 21000
    }

In [ ]:
#is_signed = input("Sign Transaction? (enter Y for yes, anything else for no) ")
if is_signed:
    signed_txn = web3_connection.eth.account.signTransaction(transaction, private_key)
    txn_hash = web3_connection.eth.sendRawTransaction(signed_txn.rawTransaction)
    link_to_etherscan = 'https://goerli.etherscan.io/tx/' + txn_hash.hex()
    print('Link to etherscan is here: ' + link_to_etherscan)

In [ ]:
#txn_hash = web3_connection.eth.sendRawTransaction(signed_txn.rawTransaction)

In [ ]:
#txn_hash
#link_to_etherscan = 'https://goerli.etherscan.io/tx/' + txn_hash.hex()
#link_to_etherscan
# https://goerli.etherscan.io/tx/0xa9f8c99ae4e7faecca79cdc93333b781fa5da0224ad63381e5f3a754fe889d72

In [ ]:
print("Your New Number of Transactions Is " + str(web3_connection.eth.getTransactionCount(wallet_address)))

In [ ]:
balance = web3_connection.eth.get_balance(account = wallet_address)
print('Your new balance is ' + str(Web3.fromWei(balance, 'ether')) + ' ether')